### Connect to back-end

In [ ]:
from bioexplorer import BioExplorer, Vector3

url = 'localhost:5000'
be = BioExplorer(url)
core = be.core_api()
status = be.reset_scene()

generate_internals = False

In [ ]:
presynaptic_neuron_id = 47211
postsynaptic_1_neuron_id = 47796


In [ ]:
import os
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from tqdm.notebook import tqdm

db_host = os.getenv('DB_HOST')
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_schema = 'o1'

db_connection_string = 'postgresql+psycopg2://%s:%s@%s:5432/%s' % (db_user, db_password, db_host, db_name)
print('Connection string: ' + db_connection_string + ', schema: ' + db_schema)

engine = create_engine(db_connection_string)
conn = engine.connect()

### Load neurons

In [ ]:
neuron_population_name = 'o1'

presynaptic_assembly_name = 'PreSynaptic Neuron'
be.remove_assembly(presynaptic_assembly_name)
neurons_assembly = be.add_assembly(presynaptic_assembly_name)
status = be.add_neurons(
    assembly_name=presynaptic_assembly_name,
    population_name=neuron_population_name,
    morphology_representation=be.MORPHOLOGY_REPRESENTATION_SECTION,
    morphology_color_scheme=be.MORPHOLOGY_COLOR_SCHEME_SECTION_TYPE,
    use_sdf=True, load_synapses=True, generate_varicosities=True,
    load_somas=True, load_axon=True, show_membrane=True,
    load_basal_dendrites=True, load_apical_dendrites=True,
    generate_internals=generate_internals, generate_externals=False,
    sql_node_filter='guid=%d' % presynaptic_neuron_id
)


In [ ]:
postsynaptic_1_assembly_name = 'PostSynaptic Neuron 1'
be.remove_assembly(postsynaptic_1_assembly_name)
neurons_assembly = be.add_assembly(postsynaptic_1_assembly_name)

status = be.add_neurons(
    assembly_name=postsynaptic_1_assembly_name,
    population_name=neuron_population_name,
    morphology_representation=be.MORPHOLOGY_REPRESENTATION_SECTION,
    morphology_color_scheme=be.MORPHOLOGY_COLOR_SCHEME_SECTION_TYPE,
    use_sdf=True, load_synapses=True, generate_varicosities=False,
    load_somas=True, load_axon=True, show_membrane=True,
    load_basal_dendrites=True, load_apical_dendrites=True,
    generate_internals=generate_internals, generate_externals=False,
    sql_node_filter='guid=%d' % (postsynaptic_1_neuron_id)
)

In [ ]:
with Session(engine) as session:
    data = session.execute('SELECT center_x_position, center_y_position, center_z_position FROM %s.synapse WHERE presynaptic_neuron_guid=%d AND postsynaptic_neuron_guid=%d' % (db_schema, presynaptic_neuron_id, postsynaptic_1_neuron_id))
    result = data.all()[1]
    target = [result[0], result[1], result[2]]
    position = [target[0], target[1], target[2] + 10]
    core.set_camera(orientation=[0, 0, 0, 1], position=position, target=target)
    # core.set_renderer(current='basic', subsampling=4)


In [ ]:
vasculature_assembly_name = 'Vasculature'
vasculature_population_name = 'vasculature'
be.remove_assembly(vasculature_assembly_name)
vasculature_assembly = be.add_assembly(vasculature_assembly_name)
vasculature_model = be.add_vasculature(
    assembly_name=vasculature_assembly_name,
    population_name=vasculature_population_name,
    representation=be.VASCULATURE_REPRESENTATION_SECTION,
    use_sdf=True,
    sql_filter='sqrt(pow(x - %f, 2) + pow(y - %f, 2) + pow(z - %f, 2)) < 200' % (target[0], target[1], target[2])
)

In [ ]:
astrocytes_assembly_name = 'Astrocytes'
be.remove_assembly(astrocytes_assembly_name)
astrocytes_assembly = be.add_assembly(astrocytes_assembly_name)
astrocytes_model = be.add_astrocytes(
    assembly_name=astrocytes_assembly_name,
    population_name=vasculature_population_name,
    # vasculature_population_name=vasculature_population_name,
    use_sdf=True, generate_internals=True,
    load_somas=True, load_dendrites=True,
    sql_filter='sqrt(pow(x - %f, 2) + pow(y - %f, 2) + pow(z - %f, 2)) < 50' % (target[0], target[1], target[2])
)

### Materials

In [ ]:
def set_morphology_materials(model_id, palette_name, palette_size, shading_mode, glossiness=1.0, emission=0.0, user_param=1.0, opacity = 1.0):
    import seaborn as sns
    colors = list()
    opacities = list()
    refraction_indices = list()
    specular_exponents = list()
    shading_modes = list()
    user_params = list()
    glossinesses = list()
    emissions = list()
    
    material_ids = be.get_material_ids(model_id)['ids']
    palette_size = len(material_ids)
    palette = list()
    intensity = 1.0
    for p in sns.color_palette(palette_name, palette_size):
        palette.append([p[0] * intensity, p[1] * intensity, p[2] * intensity])

    for material_id in material_ids:
        mid = material_id % palette_size
        colors.append(palette[mid])
        if mid in [be.NEURON_MATERIAL_AFFERENT_SYNPASE, be.NEURON_MATERIAL_EFFERENT_SYNPASE]:
            # colors.append(palette[1])
            opacities.append(opacity)
            shading_modes.append(shading_mode)
            glossinesses.append(glossiness)
            specular_exponents.append(5.0)
        elif mid in [be.NEURON_MATERIAL_VARICOSITY]:
            # colors.append(palette[min(2, palette_size-1)])
            opacities.append(opacity)
            shading_modes.append(shading_mode)
            glossinesses.append(glossiness)
            specular_exponents.append(5.0)
        elif mid == be.NEURON_MATERIAL_MITOCHONDRION:
            # colors.append([0.5, 0.1, 0.6])
            opacities.append(opacity)
            shading_modes.append(shading_mode)
            glossinesses.append(glossiness)
            specular_exponents.append(6.0)
        elif mid == be.NEURON_MATERIAL_NUCLEUS:
            # colors.append([1.0, 1.0, 1.0])
            opacities.append(opacity)
            shading_modes.append(shading_mode)
            glossinesses.append(glossiness)
            specular_exponents.append(30.0)
        elif mid == be.NEURON_MATERIAL_SOMA:
            # colors.append(palette[0])
            if generate_internals:
                opacities.append(opacity * 0.5)
                # opacities.append(opacity)
            else:
                opacities.append(opacity)
            shading_modes.append(shading_mode)
            glossinesses.append(glossiness)
            specular_exponents.append(5.0)
        elif mid == be.NEURON_MATERIAL_MYELIN_STEATH:
            # colors.append([0.4, 0.3, 0.5])
            opacities.append(opacity)
            shading_modes.append(shading_mode)
            glossinesses.append(glossiness)
            specular_exponents.append(50.0)
        else:
            # Membrane
            # colors.append(palette[0])
            opacities.append(1.0)
            shading_modes.append(shading_mode)
            glossinesses.append(glossiness)
            specular_exponents.append(5.0)
            
        refraction_indices.append(2.5)
        emissions.append(emission)
        user_params.append(user_param)
        
    be.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        diffuse_colors=colors, specular_colors=colors,
        opacities=opacities, refraction_indices=refraction_indices,
        shading_modes=shading_modes, specular_exponents=specular_exponents,
        user_parameters=user_params, glossinesses=glossinesses,
        emissions=emissions
    )

In [ ]:
def set_materials():
    palettes = ['Set3_r', 'Set1', 'Reds_r', 'GnBu']
    model_ids = be.get_model_ids()['ids']
    i = 0
    user_param = 0.01
    for i in range(len(model_ids)):
        model_id = model_ids[i]
        if i==2:
            # Vasculature
            set_morphology_materials(
                model_id, palettes[i], be.NB_MATERIALS_PER_MORPHOLOGY,
                be.SHADING_MODE_PERLIN, 0.1, 0.0, 0.01, 1.0)
                # be.SHADING_MODE_CARTOON, 1.0, 0.0, 3.0)
                # be.SHADING_MODE_NONE, 1.0, 0.0, 3.0)
        else:
            set_morphology_materials(
                model_id, palettes[i], be.NB_MATERIALS_PER_MORPHOLOGY,
                be.SHADING_MODE_PERLIN, 0.1, 0.0, user_param)
                # be.SHADING_MODE_CARTOON, 1.0, 0.0, 3.0)
                # be.SHADING_MODE_NONE, 1.0, 0.0, 3.0)
        i += 1
    core.set_renderer()

set_materials()

In [ ]:
status = core.set_renderer(
    background_color=[0.18, 0.43, 0.41], head_light=True,
    current='bio_explorer',subsampling=4, max_accum_frames=64)
params = core.BioExplorerRendererParams()
params.fog_start = 100
params.fog_thickness = 100
params.gi_samples = 1
params.gi_weight = 0.1
params.gi_distance = 20.0
params.shadows = 0.8
params.soft_shadows = 1.0
params.epsilon_factor = 1.0
params.exposure = 1.0
params.max_bounces = 1
params.show_background = True
params.use_hardware_randomizer=False
status = core.set_renderer_params(params)
status = core.set_renderer()